In [1]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_API_KEY"] = "38002279fa574fbcba0fbc424ae056d5"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://lumin-azure-openai-rnd.openai.azure.com/"

model = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment='lumin-rnd-gpt-4-32k',
)

In [3]:
from langchain import LLMChain
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)


llm_chain = LLMChain(
    prompt=long_prompt,
    llm=model
)

qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

res = llm_chain.generate(qs)
res

/tmp/pip_packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


1. The Green Bay Packers won the Super Bowl in the 2010 season.
2. If you are 6 ft 4 inches tall, you are approximately 193 centimeters tall.
3. There has not been a 12th person on the moon, as only 12 astronauts have walked on the moon, with Eugene Cernan being the last (11th) person to do so.
4. A blade of grass does not have any eyes.
